**Table of contents**<a id='toc0_'></a>    
- [Libraries](#toc1_)    
- [Find current companies with CIK codes](#toc2_)    
- [Find basic information on companies](#toc3_)    
- [Download the index files listing all filings by year and quarter](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Libraries](#toc0_)

In [1]:
import os
import io
import glob
import requests
import pandas as pd

# <a id='toc2_'></a>[Find current companies with CIK codes](#toc0_)

In [2]:
os.chdir('..')
agent = os.environ.get("AGENT")
from secnlp import data_sourcing as ds

In [3]:
companies_list = ds.current_edgar_companies_list(agent = agent)
print(companies_list)

           ticker                                name
cik                                                  
0000320193   AAPL                          Apple inc.
0000789019   MSFT                      Microsoft corp
0001652044  GOOGL                       Alphabet inc.
0001018724   AMZN                      Amazon com inc
0001045810   NVDA                         Nvidia corp
...           ...                                 ...
0001765651   PSPX        Pacific sports exchange inc.
0001114446   AMNA                              Ubs ag
0001887912   WLSS                        Welsis corp.
0001919191   PGDE                   Pgd eco solutions
0001788756   PFTY  Parallel flight technologies, inc.

[8226 rows x 2 columns]


# <a id='toc3_'></a>[Find basic information on companies](#toc0_)

In [4]:
cik_list = companies_list.index.values[:10]
companies_info = ds.basic_info_company(cik_list=cik_list, agent = agent)
print(companies_info)

          sic                                     sicDescription tickers  \
cik                                                                        
320193   3571                               Electronic Computers    AAPL   
789019   7372                      Services-Prepackaged Software    MSFT   
1652044  7370  Services-Computer Programming, Data Processing...   GOOGL   
1018724  5961                 Retail-Catalog & Mail-Order Houses    AMZN   
1045810  3674                   Semiconductors & Related Devices    NVDA   
1326801  7370  Services-Computer Programming, Data Processing...    META   
1318605  3711              Motor Vehicles & Passenger Car Bodies    TSLA   
1067983  6331                  Fire, Marine & Casualty Insurance   BRK-B   
59478    2834                        Pharmaceutical Preparations     LLY   
1046179  3674                   Semiconductors & Related Devices     TSM   

        exchanges fiscalYearEnd  
cik                              
320193     Nasdaq  

# <a id='toc4_'></a>[Download the index files listing all filings by year and quarter](#toc0_)

In [3]:
filings = ds.bulk_download_url_filings(start_year = 1993, end_year = 1994, quarters = ['QTR1','QTR2','QTR3','QTR4'],
                              agent = agent, uncompress = True)

,cik,company,form_type,date_filed,file_name
0,0000860585,RBS PARTNERS L P /CT,13FCONP,1993-05-12,edgar/data/860585/9999999997-04-042068.txt
1,0000880794,MERRILL LYNCH LIFE VARIABLE ANNUITY SEPARATE A...,POS AMI,1993-04-28,edgar/data/880794/9999999997-05-050434.txt
2,0000926688,SMITH THOMAS W,13F-HR,1993-05-14,edgar/data/926688/9999999997-05-015657.txt
3,0000926688,SMITH THOMAS W,13FCONP,1993-05-14,edgar/data/926688/9999999997-05-015662.txt
0,0000060512,LOUISIANA LAND & EXPLORATION CO,10-Q,1993-08-13,edgar/data/60512/0000060512-94-000005.txt
...,...,...,...,...,...
20878,0000099974,TJ INTERNATIONAL INC,10-K,1994-03-29,edgar/data/99974/0000912057-94-001108.txt
0,0000860585,RBS PARTNERS L P /CT,13FCONP,1993-02-11,edgar/data/860585/9999999997-04-035713.txt
1,0000880794,MERRILL LYNCH LIFE VARIABLE ANNUITY SEPARATE A...,NSAR-B,1993-02-26,edgar/data/880794/9999999997-05-050433.txt
2,0000926688,SMITH THOMAS W,13F-HR,1993-02-12,edgar/data/926688/9999999997-05-015654.txt
